## Notebook Goals

Note: this is a [unfinished] side path trying to find optimal model. This notebook is not apart of main solution

The goal of this notebook is to fit and find the best model for business case

## Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.feature_selection import f_classif

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

from IPython.display import display
pd.set_option('display.max_columns', None)

from sklearn.neural_network import MLPRegressor

In [2]:
df = pd.read_csv('../data/amzn_final_dataset.csv' ,index_col='AMZN')
df.dropna(inplace=True)

In [3]:
df = df.iloc[-500:]
df

,Open,High,Low,Close,Volume,r_percent_increase_high,r_percent_increase_close,c_four_percent_high,SMA,SMA_diff,Stochastic,Stochastic_diff,rsi_up_var,rsi_down_var,RSI,RSI_diff,ROC,ROC_diff,tr,ATR,ATR_diff,pos_dx,neg_dx,dx,ADX,ADX_diff,rsi_over_80,rsi_over_70,rsi_over_60,rsi_under_40,rsi_under_30,rsi_under_20,sto_over_80,sto_over_70,sto_over_60,sto_under_40,sto_under_30,sto_under_20
AMZN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-02-15,1627.86,1628.9100,1604.50,1607.95,4343893.0,2.901210,1.557884,0,1622.363333,-2.817778,24.279399,-8.664898,0.00,14.70,46.661165,1.799047,-1.552675,-1.332534,24.4100,46.014286,-2.290714,0.00,0.00,41.299223,26.966062,-0.604649,0,0,0,0,0,0,0,0,0,1,1,1
2019-02-19,1601.00,1634.0000,1600.56,1627.58,3681656.0,1.660133,0.541909,0,1618.893333,-3.470000,35.850280,11.570881,19.63,0.00,53.974244,7.313079,-1.882675,-0.330000,33.4400,45.033571,-0.980714,5.09,0.00,42.575272,24.074783,-2.891279,0,0,0,0,0,0,0,0,0,1,0,0
2019-02-20,1630.00,1634.9300,1610.12,1622.10,3337589.0,2.003576,1.170705,0,1616.875556,-2.017778,32.620100,-3.230180,0.00,5.48,43.152645,-10.821599,-1.107142,0.775533,24.8100,40.872143,-4.161429,0.00,9.56,22.878377,22.709063,-1.365721,0,0,0,0,0,0,0,0,0,1,0,0
2019-02-21,1619.85,1623.5600,1600.91,1619.44,3483392.0,2.171121,1.336882,0,1617.438889,0.563333,49.557855,16.937755,0.00,2.66,33.843200,-9.309445,0.314054,1.421196,22.6500,37.777143,-3.095000,0.00,0.00,9.951317,22.666787,-0.042276,0,0,0,1,0,0,0,0,0,0,0,0
2019-02-22,1623.50,1634.9400,1621.17,1631.56,3096191.0,2.617127,2.462061,0,1622.254444,4.815556,61.421801,11.863946,12.12,0.00,51.174578,17.331378,2.728841,2.414787,15.5000,31.975714,-5.801429,0.00,20.26,21.358009,23.806821,1.140035,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-03,3425.01,3434.0000,3308.62,3312.53,7088781.0,1.946247,1.196065,0,3291.574444,0.615556,64.150164,-21.494964,0.00,67.47,60.714286,-8.863149,0.167524,-3.406072,125.3800,88.029586,4.018571,6.26,0.00,0.867212,17.325803,-0.145051,0,0,1,0,0,0,0,0,1,0,0,0
2021-02-04,3330.00,3347.0000,3277.75,3331.00,3670661.0,1.380967,0.634944,0,3295.882222,4.307778,69.601275,5.451111,18.47,0.00,65.317519,4.603233,1.177621,1.010097,69.2500,88.875307,0.845721,0.00,0.00,0.867212,17.273923,-0.051880,0,0,1,0,0,0,0,0,1,0,0,0
2021-02-05,3319.00,3377.0000,3302.71,3352.15,3620808.0,0.383333,-0.871381,0,3302.343333,6.461111,75.784024,6.182749,21.15,0.00,68.715084,3.397565,1.765331,0.587710,74.2900,90.797807,1.922500,30.00,0.00,5.519249,17.554331,0.280408,0,0,1,0,0,0,0,1,1,0,0,0


## Train/Test Split

In [4]:
y = df['r_percent_increase_close']
x = df[['SMA', 'Stochastic', 'RSI', 'ROC', 'ATR', 'ADX']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 123)

In [5]:
mm_scaler = MinMaxScaler()
mm_scaler.fit(x_train)

x_train = mm_scaler.transform(x_train)
x_test = mm_scaler.transform(x_test)

## Modeling

### Function

In [6]:
def rmse(model, xtrain, ytrain, xtest, ytest):
    yhat_train = model.predict(xtrain)
    rmse_train = np.sqrt(metrics.mean_squared_error(ytrain, yhat_train))

    yhat_test = model.predict(xtest)
    rmse_test = np.sqrt(metrics.mean_squared_error(ytest, yhat_test))

    print('RMSE Train:', rmse_train)
    print('RMSE Test:', rmse_test)

### Decision Tree Regression

In [7]:
dtr = DecisionTreeRegressor(criterion='mse',
                            max_depth=2,
                            max_features='log2',
                            min_samples_leaf=3,
                            min_samples_split=5)
dtr.fit(x_train, y_train)

rmse(dtr, x_train, y_train, x_test, y_test)

RMSE Train: 2.9311277635978845
RMSE Test: 3.2774577421651654


### SVR

In [8]:
svr = SVR(C=.001, degree=1, gamma=0.001, kernel='rbf')
# svr.fit(scaled_x_train, y_train)
svr.fit(x_train, y_train)
# rmse(svr, scaled_x_train, y_train, scaled_x_test, y_test)
rmse(svr, x_train, y_train, x_test, y_test)

RMSE Train: 3.2303389158841327
RMSE Test: 3.334975788920539


### KNR

In [9]:
knr = KNeighborsRegressor(algorithm='auto', metric='manhattan', n_neighbors=19, weights='distance')
# knr.fit(scaled_x_train, y_train)
# rmse(knr, scaled_x_train, y_train, scaled_x_test, y_test)

knr.fit(x_train, y_train)
rmse(knr, x_train, y_train, x_test, y_test)

RMSE Train: 0.0
RMSE Test: 2.5618670286122858


### Random Forest Regressor

In [10]:
rfr = RandomForestRegressor(criterion='mae', max_depth=30, max_features='sqrt', min_samples_leaf=10, min_samples_split=5, n_estimators=20)
rfr.fit(x_train, y_train)
rmse(rfr, x_train, y_train, x_test, y_test)

RMSE Train: 2.667937073925964
RMSE Test: 2.947485259617937


### AdaBoost Regressor

In [11]:
ada_reg = AdaBoostRegressor(learning_rate=.05, loss='square', n_estimators=100)
ada_reg.fit(x_train, y_train)
rmse(ada_reg, x_train, y_train, x_test, y_test)

RMSE Train: 2.581139381702498
RMSE Test: 2.95417828231247


###  Gradient Boosting Regressor

In [12]:
grad_boost_reg = GradientBoostingRegressor(criterion= 'mse', learning_rate= 0.001, loss= 'huber', n_estimators = 150)
grad_boost_reg.fit(x_train, y_train)
rmse(grad_boost_reg, x_train, y_train, x_test, y_test)

RMSE Train: 3.1366165849114505
RMSE Test: 3.270174399122882
